# Model Trainer

In [1]:
import os
os.chdir("../")
%pwd

'/home/migue/data-science/portfolio/text-summarizer'

## Entity

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: str
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [3]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
    ):

        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self):
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            per_device_eval_batch_size=params.per_device_eval_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=params.eval_steps,
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps,
        )

        return model_trainer_config

## Components

In [5]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_from_disk
import torch

[2024-09-02 19:02:09,975: INFO: config: PyTorch version 2.3.1 available.]


In [6]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(
            self.config.model_ckpt
        ).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        dataset_pt = load_from_disk(self.config.data_path)

        args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=self.config.per_device_eval_batch_size,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy,
            eval_steps=self.config.eval_steps,
            save_steps=self.config.save_steps,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps,
        )

        trainer = Trainer(
            model=model_pegasus,
            args=args,
            tokenizer=tokenizer,
            data_collator=seq2seq_data_collator,
            train_dataset=dataset_pt["train"],
            eval_dataset=dataset_pt["validation"],
        )

        trainer.train()

        model_pegasus.save_pretrained(
            os.path.join(self.config.root_dir, "pegasus-samsum-model")
        )
        tokenizer.save_pretrained(
            os.path.join(self.config.root_dir, "pegasus-samsum-model")
        )

## Pipeline

In [7]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-09-02 19:02:10,108: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-09-02 19:02:10,109: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-02 19:02:10,110: INFO: common: created a directory at: artifacts]
[2024-09-02 19:02:10,110: INFO: common: created a directory at: artifacts/model_trainer]


/home/migue/miniconda3/envs/pytorch/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/migue/miniconda3/envs/pytorch/lib/python3.12/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  wa

  0%|          | 0/1227 [00:00<?, ?it/s]

{'loss': 3.0899, 'grad_norm': 11.069536209106445, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}
{'loss': 2.9334, 'grad_norm': 25.39984130859375, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.02}
{'loss': 3.1411, 'grad_norm': 12.329587936401367, 'learning_rate': 3e-06, 'epoch': 0.02}
{'loss': 3.0877, 'grad_norm': 12.36679744720459, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.03}
{'loss': 3.08, 'grad_norm': 10.263320922851562, 'learning_rate': 5e-06, 'epoch': 0.04}
{'loss': 2.7008, 'grad_norm': 37.445091247558594, 'learning_rate': 6e-06, 'epoch': 0.05}
{'loss': 2.8397, 'grad_norm': 34.78185272216797, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.06}
{'loss': 2.6756, 'grad_norm': 7.89933967590332, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.07}
{'loss': 2.365, 'grad_norm': 13.456506729125977, 'learning_rate': 9e-06, 'epoch': 0.07}
{'loss': 2.3218, 'grad_norm': 11.648741722106934, 'learning_rate': 1e-05, 'epoch': 0.08}
{'loss': 2.3053, 'grad_norm': 6.6981

  0%|          | 0/205 [00:00<?, ?it/s]

{'eval_loss': 1.5456265211105347, 'eval_runtime': 11.1103, 'eval_samples_per_second': 73.625, 'eval_steps_per_second': 18.451, 'epoch': 0.41}
{'loss': 1.7593, 'grad_norm': 3.390408992767334, 'learning_rate': 4.931224209078404e-05, 'epoch': 0.42}
{'loss': 1.6637, 'grad_norm': 6.170805931091309, 'learning_rate': 4.862448418156809e-05, 'epoch': 0.42}
{'loss': 1.7241, 'grad_norm': 6.4096269607543945, 'learning_rate': 4.793672627235214e-05, 'epoch': 0.43}
{'loss': 1.6482, 'grad_norm': 5.202908039093018, 'learning_rate': 4.724896836313618e-05, 'epoch': 0.44}
{'loss': 1.7453, 'grad_norm': 4.602560043334961, 'learning_rate': 4.6561210453920225e-05, 'epoch': 0.45}
{'loss': 1.6246, 'grad_norm': 4.560962677001953, 'learning_rate': 4.5873452544704265e-05, 'epoch': 0.46}
{'loss': 1.7239, 'grad_norm': 7.0066070556640625, 'learning_rate': 4.518569463548831e-05, 'epoch': 0.46}
{'loss': 1.7155, 'grad_norm': 5.484926223754883, 'learning_rate': 4.449793672627235e-05, 'epoch': 0.47}
{'loss': 1.6879, 'grad

  0%|          | 0/205 [00:00<?, ?it/s]

{'eval_loss': 1.4777475595474243, 'eval_runtime': 11.2453, 'eval_samples_per_second': 72.741, 'eval_steps_per_second': 18.23, 'epoch': 0.81}
{'loss': 1.5978, 'grad_norm': 7.776360511779785, 'learning_rate': 1.4924346629986246e-05, 'epoch': 0.82}
{'loss': 1.5762, 'grad_norm': 19.00647735595703, 'learning_rate': 1.4236588720770288e-05, 'epoch': 0.83}
{'loss': 1.6485, 'grad_norm': 3.2901852130889893, 'learning_rate': 1.3548830811554334e-05, 'epoch': 0.84}
{'loss': 1.6079, 'grad_norm': 4.7837958335876465, 'learning_rate': 1.2861072902338378e-05, 'epoch': 0.85}
{'loss': 1.5617, 'grad_norm': 4.614064693450928, 'learning_rate': 1.2173314993122421e-05, 'epoch': 0.86}
{'loss': 1.6639, 'grad_norm': 5.476214408874512, 'learning_rate': 1.1485557083906467e-05, 'epoch': 0.86}
{'loss': 1.6019, 'grad_norm': 4.263691425323486, 'learning_rate': 1.0797799174690509e-05, 'epoch': 0.87}
{'loss': 1.5935, 'grad_norm': 3.207488775253296, 'learning_rate': 1.0110041265474554e-05, 'epoch': 0.88}
{'loss': 1.568, '

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


{'train_runtime': 716.1513, 'train_samples_per_second': 20.571, 'train_steps_per_second': 1.713, 'train_loss': 1.8124701065461424, 'epoch': 1.0}
